## EMA+Strategy+Backtesting+(c)

In [1]:
import yfinance as yf
import datetime as dt
import pandas as pd 
import numpy as np
import cufflinks as cf
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
import plotly.graph_objects as go
from plotly.subplots import make_subplots

init_notebook_mode(connected = True)
cf.go_offline()

In [2]:
def calculate_ema(prices, days, smoothing=2):
    ema = [sum(prices[:days]) / days]
    for price in prices[days:]:
        ema.append((price * (smoothing / (1 + days))) + ema[-1] * (1 - (smoothing / (1 + days))))
    return ema

In [3]:
def ema_backtesting(ticker, parameter, previous_days):
    
    start = dt.datetime.today()-dt.timedelta(previous_days)
    end = dt.datetime.today()
    global ohlc_intraday
    ohlc_intraday = {}
    ohlc_intraday[ticker] = yf.download(ticker,start,end)
    ohlc_intraday[ticker] = ohlc_intraday[ticker].reset_index(drop = False)
    df_signal = ohlc_intraday[ticker].copy()
    df_signal = df_signal[['Date','Close']]

    ema=pd.DataFrame(calculate_ema(df_signal["Close"],parameter),index=np.arange(parameter,len(df_signal["Close"])+1),columns=['ema'])
    df_signal = pd.concat([df_signal,ema],axis=1,join='outer')[:-1]
    condition = df_signal['Close'] > df_signal['ema']
    df_signal['Hold']=np.where(condition, 'Hold', '')
    df_signal['Hold_t-1']=df_signal['Hold'].shift(1)
    df_signal['Hold_t-2']=df_signal['Hold'].shift(2)

    buy_condition = (df_signal['Hold_t-2'] =='') & (df_signal['Hold_t-1'] =='Hold') 
    df_signal['Buy']=np.where(buy_condition, 'Buy', '')

    sell_condition = (df_signal['Hold_t-2'] =='Hold') & (df_signal['Hold_t-1'] =='') 
    df_signal['Sell']=np.where(sell_condition, 'Sell', '')
    df_signal = df_signal[['Date','Buy','Sell']]
    condition = (df_signal['Buy']=='Buy') | (df_signal['Sell']=='Sell')
    df_signal = df_signal[condition]
    df_signal = df_signal.reset_index(drop = True)
    
    columns_Trading = ['Date_Buy','Price_Buy','Date_Sell','Price_Sell','PL(%)','Holding_Days','Accumulated profit']
    global df_Trading
    df_Trading = pd.DataFrame(columns = columns_Trading)
    
    Accu_profit = 0
    for i in range(0,len(df_signal),2):
        if(i<len(df_signal)-1):
            if (df_signal['Buy'][i]=='Buy') & (df_signal['Sell'][i+1]=='Sell'):

                Date_Buy = df_signal['Date'][i]
                Date_Sell = df_signal['Date'][i+1]
                Price_Buy = ohlc_intraday[ticker][ohlc_intraday[ticker]['Date']==Date_Buy][['Open']].values[0][0]
                Price_Sell = ohlc_intraday[ticker][ohlc_intraday[ticker]['Date']==Date_Sell][['Open']].values[0][0]
                fee = 0.002 * (Price_Buy + Price_Sell)
                Holding_Days = abs(Date_Sell-Date_Buy).days
                PL = (Price_Sell-Price_Buy-fee)*100/Price_Buy
                Accu_profit += PL

                Add_Trading = pd.DataFrame([[Date_Buy,Price_Buy,Date_Sell,Price_Sell,PL,Holding_Days,Accu_profit]],columns = columns_Trading)
                df_Trading = df_Trading.append(Add_Trading)

    df_Trading = df_Trading.reset_index(drop = True)
    df_Trading = df_Trading.round(2)
    
    total_profit = df_Trading['Accumulated profit'][len(df_Trading)-1]
    cagr = ((((total_profit+100)/100)**(1/(len(ohlc_intraday[ticker])/250)))-1)*100
    win_trade = df_Trading[df_Trading["PL(%)"]>0].count()["PL(%)"]
    loss_trade =  df_Trading[df_Trading["PL(%)"]<=0].count()["PL(%)"]
    win_rate = win_trade*100/(win_trade+loss_trade)
    avg_gain = df_Trading[df_Trading["PL(%)"]>0]["PL(%)"].mean()
    avg_loss = df_Trading[df_Trading["PL(%)"]<=0]["PL(%)"].mean()
    expected_return = (win_rate*avg_gain/100)+((1-win_rate)*avg_loss/100)
    list_ = [["Total Profit",round(total_profit,2)],
            ["CAGR",str(round(cagr,2))+'%'],
             ["Win Trade",win_trade],["Loss Trade ",loss_trade],
             ["Win Rate",str(round(win_rate,2))+'%'],["Average Gain",str(round(avg_gain,2))+'%'],
             ["Average loss",str(round(avg_loss,2))+'%'],["Expected Return",str(round(expected_return,2))+'%']]
                      
    global df_summary
    df_summary = pd.DataFrame(list_,columns=['Measurement','Result'])
    
def plot_ema():
    df_plt = ohlc_intraday[ticker].copy()
    ema=pd.DataFrame(calculate_ema(df_plt["Close"],parameter),index=np.arange(parameter,len(df_plt["Close"])+1),columns=['ema'])
    df_plt = pd.concat([df_plt,ema],axis=1,join='outer')[:-1]
    df_plt['Date'] = pd.to_datetime(df_plt['Date'], format='%d-%m-%Y')

    fig = make_subplots(rows=3, cols=1,shared_xaxes=True, vertical_spacing=0.04,
    specs=[[{"type": "scatter"}],
          [{"type": "table"}],[{"type": "table"}]])

    buy = df_Trading[['Date_Buy','Price_Buy']].copy()
    sell = df_Trading[['Date_Sell','Price_Sell']].copy()
    
    fig.add_trace(go.Candlestick(x=df_plt['Date'],
                open=df_plt['Open'],
                high=df_plt['High'],
                low=df_plt['Low'],
                close=df_plt['Close']),row=1,col=1)

    fig.add_trace(go.Scatter(x = df_plt['Date'], y=df_plt['ema'],mode='lines',name='ema'+str(parameter)),row= 1,col =1)
    
    fig.add_trace(go.Scatter(x = buy['Date_Buy'], y=buy['Price_Buy'],mode='markers',name='buy'),row= 1,col =1)
    fig.add_trace(go.Scatter(x = sell['Date_Sell'], y=sell['Price_Sell'],mode='markers',name='sell'),row= 1,col =1)
    
    fig.add_trace(go.Table(header=dict(values=df_summary.columns,font=dict(size=10),align="left"),
            cells=dict(values=[df_summary[k].tolist() for k in df_summary.columns],align = "left")),row=2, col=1)

    fig.add_trace(go.Table(header=dict(values=df_Trading.columns,font=dict(size=10),align="left"),
            cells=dict(values=[df_Trading[k].tolist() for k in df_Trading.columns],align = "left")),row=3, col=1)

    fig.update_layout( height=900, showlegend=True, title_text='EMA',)
    fig.update_layout(xaxis_rangeslider_visible=False)
    
    fig.show()

In [4]:
ticker ='AMZN'
parameter = 40
ema_backtesting(ticker, parameter = parameter, previous_days = 4000)

[*********************100%***********************]  1 of 1 completed


In [5]:
plot_ema()

In [6]:
# 1. Download Date from yfinance
ticker = 'AAPL'
start = dt.datetime.today()-dt.timedelta(3000)
end = dt.datetime.today()
ohlc_intraday = {}
ohlc_intraday[ticker] = yf.download(ticker,start,end)
ohlc_intraday[ticker] = ohlc_intraday[ticker].reset_index(drop = False)
df_signal = ohlc_intraday[ticker].copy()
df_signal

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2013-09-30,17.044643,17.202143,16.943214,17.026787,15.010161,260156400
1,2013-10-01,17.087500,17.469286,17.084999,17.427143,15.363092,353883600
2,2013-10-02,17.343929,17.564285,17.276787,17.484285,15.413467,289184000
3,2013-10-03,17.518213,17.583929,17.169287,17.264643,15.219851,322753200
4,2013-10-04,17.280714,17.307142,17.092857,17.251072,15.207886,258868400
...,...,...,...,...,...,...,...
2065,2021-12-10,175.210007,179.630005,174.690002,179.449997,179.449997,115228100
2066,2021-12-13,181.119995,182.130005,175.529999,175.740005,175.740005,153237000
2067,2021-12-14,175.250000,177.740005,172.210007,174.330002,174.330002,139380400
2068,2021-12-15,175.110001,179.500000,172.309998,179.300003,179.300003,131063300


In [7]:
# 2.remove some columns
df_signal = df_signal[['Date','Close']]
df_signal

,Date,Close
0,2013-09-30,17.026787
1,2013-10-01,17.427143
2,2013-10-02,17.484285
3,2013-10-03,17.264643
4,2013-10-04,17.251072
...,...,...
2065,2021-12-10,179.449997
2066,2021-12-13,175.740005
2067,2021-12-14,174.330002
2068,2021-12-15,179.300003


In [8]:
# 3.Add ema
parameter = 40
ema=pd.DataFrame(calculate_ema(df_signal["Close"],parameter)
                 ,index=np.arange(parameter,len(df_signal["Close"])+1),columns=['ema'])
df_signal = pd.concat([df_signal,ema],axis=1,join='outer')[:-1]
df_signal

,Date,Close,ema
0,2013-09-30,17.026787,NaN
1,2013-10-01,17.427143,NaN
2,2013-10-02,17.484285,NaN
3,2013-10-03,17.264643,NaN
4,2013-10-04,17.251072,NaN
...,...,...,...
2065,2021-12-10,179.449997,157.221829
2066,2021-12-13,175.740005,158.306129
2067,2021-12-14,174.330002,159.156562
2068,2021-12-15,179.300003,159.896730


In [9]:
# 4. Find days 'hold'
condition = df_signal['Close'] > df_signal['ema']
df_signal['Hold']=np.where(condition, 'Hold', '')
df_signal

,Date,Close,ema,Hold
0,2013-09-30,17.026787,NaN,
1,2013-10-01,17.427143,NaN,
2,2013-10-02,17.484285,NaN,
3,2013-10-03,17.264643,NaN,
4,2013-10-04,17.251072,NaN,
...,...,...,...,...
2065,2021-12-10,179.449997,157.221829,Hold
2066,2021-12-13,175.740005,158.306129,Hold
2067,2021-12-14,174.330002,159.156562,Hold
2068,2021-12-15,179.300003,159.896730,Hold


In [10]:
# 5. Find 'buy','sell' day

df_signal['Hold_t-1']=df_signal['Hold'].shift(1)
df_signal['Hold_t-2']=df_signal['Hold'].shift(2)

buy_condition = (df_signal['Hold_t-2'] =='') & (df_signal['Hold_t-1'] =='Hold') 
df_signal['Buy']=np.where(buy_condition, 'Buy', '')

sell_condition = (df_signal['Hold_t-2'] =='Hold') & (df_signal['Hold_t-1'] =='') 
df_signal['Sell']=np.where(sell_condition, 'Sell', '')

df_signal

,Date,Close,ema,Hold,Hold_t-1,Hold_t-2,Buy,Sell
0,2013-09-30,17.026787,NaN,,NaN,NaN,,
1,2013-10-01,17.427143,NaN,,,NaN,,
2,2013-10-02,17.484285,NaN,,,,,
3,2013-10-03,17.264643,NaN,,,,,
4,2013-10-04,17.251072,NaN,,,,,
...,...,...,...,...,...,...,...,...
2065,2021-12-10,179.449997,157.221829,Hold,Hold,Hold,,
2066,2021-12-13,175.740005,158.306129,Hold,Hold,Hold,,
2067,2021-12-14,174.330002,159.156562,Hold,Hold,Hold,,
2068,2021-12-15,179.300003,159.896730,Hold,Hold,Hold,,


In [11]:
# 6. remove some columns
df_signal = df_signal[['Date','Buy','Sell']]
df_signal

,Date,Buy,Sell
0,2013-09-30,,
1,2013-10-01,,
2,2013-10-02,,
3,2013-10-03,,
4,2013-10-04,,
...,...,...,...
2065,2021-12-10,,
2066,2021-12-13,,
2067,2021-12-14,,
2068,2021-12-15,,


In [12]:
# 7. filter days sending order
condition = (df_signal['Buy']=='Buy') | (df_signal['Sell']=='Sell')

df_signal = df_signal[condition]
df_signal = df_signal.reset_index(drop = True)
df_signal

,Date,Buy,Sell
0,2013-11-26,Buy,
1,2014-01-06,,Sell
2,2014-01-15,Buy,
3,2014-01-21,,Sell
4,2014-01-22,Buy,
...,...,...,...
152,2021-06-14,Buy,
153,2021-09-15,,Sell
154,2021-09-16,Buy,
155,2021-09-20,,Sell


In [13]:
# 8. Make trading dataframe
columns_Trading = ['Date_Buy','Price_Buy','Date_Sell'
                   ,'Price_Sell','PL(%)','Holding_Days'
                   ,'Accumulated profit']
df_Trading = pd.DataFrame(columns = columns_Trading)
df_Trading

,Date_Buy,Price_Buy,Date_Sell,Price_Sell,PL(%),Holding_Days,Accumulated profit


In [14]:
# 9. fill data in trading dataframe

Accu_profit = 0
for i in range(0,len(df_signal),2):
    if(i<len(df_signal)-1):
        if (df_signal['Buy'][i]=='Buy') & (df_signal['Sell'][i+1]=='Sell'):

            Date_Buy = df_signal['Date'][i]
            Date_Sell = df_signal['Date'][i+1]
            Price_Buy = ohlc_intraday[ticker][ohlc_intraday[ticker]['Date']==Date_Buy][['Open']].values[0][0]
            Price_Sell = ohlc_intraday[ticker][ohlc_intraday[ticker]['Date']==Date_Sell][['Open']].values[0][0]
            fee = 0.002 * (Price_Buy + Price_Sell)
            Holding_Days = abs(Date_Sell-Date_Buy).days
            PL = (Price_Sell-Price_Buy-fee)*100/Price_Buy
            Accu_profit += PL

            Add_Trading = pd.DataFrame([[Date_Buy,Price_Buy,Date_Sell,Price_Sell,PL,Holding_Days,Accu_profit]]
                                       ,columns = columns_Trading)
            df_Trading = df_Trading.append(Add_Trading)

df_Trading = df_Trading.reset_index(drop = True)
df_Trading = df_Trading.round(2)
df_Trading

,Date_Buy,Price_Buy,Date_Sell,Price_Sell,PL(%),Holding_Days,Accumulated profit
0,2013-11-26,18.72,2014-01-06,19.19,2.14,41,2.14
1,2014-01-15,19.77,2014-01-21,19.32,-2.66,6,-0.52
2,2014-01-22,19.68,2014-01-29,18.00,-8.91,7,-9.43
3,2014-02-12,19.18,2014-02-21,19.03,-1.17,9,-10.60
4,2014-03-06,19.03,2014-03-07,18.97,-0.72,1,-11.32
...,...,...,...,...,...,...,...
73,2021-04-06,126.50,2021-05-05,129.20,1.73,29,154.67
74,2021-05-07,130.85,2021-05-11,123.50,-6.01,4,148.67
75,2021-06-10,127.02,2021-06-11,126.53,-0.78,1,147.88
76,2021-06-14,127.82,2021-09-15,148.56,15.79,93,163.68


In [15]:
#10. Trading Summary
total_profit = df_Trading['Accumulated profit'][len(df_Trading)-1]
cagr = ((((total_profit+100)/100)**(1/(len(ohlc_intraday[ticker])/250)))-1)*100
win_trade = df_Trading[df_Trading["PL(%)"]>0].count()["PL(%)"]
loss_trade =  df_Trading[df_Trading["PL(%)"]<=0].count()["PL(%)"]
win_rate = win_trade*100/(win_trade+loss_trade)
avg_gain = df_Trading[df_Trading["PL(%)"]>0]["PL(%)"].mean()
avg_loss = df_Trading[df_Trading["PL(%)"]<=0]["PL(%)"].mean()
expected_return = (win_rate*avg_gain/100)+((1-win_rate)*avg_loss/100)
list_ = [["Total Profit",round(total_profit,2)],
            ["CAGR",str(round(cagr,2))+'%'],
             ["Win Trade",win_trade],["Loss Trade ",loss_trade],
             ["Win Rate",str(round(win_rate,2))+'%'],
             ["Average Gain",str(round(avg_gain,2))+'%'],
             ["Average loss",str(round(avg_loss,2))+'%'],
             ["Expected Return",str(round(expected_return,2))+'%']]
                      
df_summary = pd.DataFrame(list_,columns=['Measurement','Result'])
df_summary

,Measurement,Result
0,Total Profit,160.16
1,CAGR,12.24%
2,Win Trade,25
3,Loss Trade,53
4,Win Rate,32.05%
5,Average Gain,10.9%
6,Average loss,-2.12%
7,Expected Return,4.15%


In [16]:
#11.Plot graph

df_plt = ohlc_intraday[ticker].copy()
ema=pd.DataFrame(calculate_ema(df_plt["Close"],parameter),index=np.arange(parameter,len(df_plt["Close"])+1),columns=['ema'])
df_plt = pd.concat([df_plt,ema],axis=1,join='outer')[:-1]
df_plt['Date'] = pd.to_datetime(df_plt['Date'], format='%d-%m-%Y')

fig = make_subplots(rows=3, cols=1,shared_xaxes=True, vertical_spacing=0.04,
specs=[[{"type": "scatter"}],
      [{"type": "table"}],[{"type": "table"}]])

buy = df_Trading[['Date_Buy','Price_Buy']].copy()
sell = df_Trading[['Date_Sell','Price_Sell']].copy()

fig.add_trace(go.Candlestick(x=df_plt['Date'],open=df_plt['Open'],high=df_plt['High'],low=df_plt['Low'],
                             close=df_plt['Close']),row=1,col=1)

fig.add_trace(go.Scatter(x = df_plt['Date'], y=df_plt['ema'],mode='lines',name='ema'+str(parameter)),row= 1,col =1)
fig.add_trace(go.Scatter(x = buy['Date_Buy'], y=buy['Price_Buy'],mode='markers',name='buy'),row= 1,col =1)
fig.add_trace(go.Scatter(x = sell['Date_Sell'], y=sell['Price_Sell'],mode='markers',name='sell'),row= 1,col =1)

fig.add_trace(go.Table(header=dict(values=df_summary.columns,font=dict(size=10),align="left"),
        cells=dict(values=[df_summary[k].tolist() for k in df_summary.columns],align = "left")),row=2, col=1)

fig.add_trace(go.Table(header=dict(values=df_Trading.columns,font=dict(size=10),align="left"),
        cells=dict(values=[df_Trading[k].tolist() for k in df_Trading.columns],align = "left")),row=3, col=1)

fig.update_layout( height=900, showlegend=True, title_text='EMA',)
fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()